# Align Faces
by @duskvirkus
Thanks to Derrick Schultz for the orginal notebook.

This notebook will walk thru the process to take a folder of face images and convert them to images in the format of FFHQ.

In [ ]:
# notebook utils
import os
root_dir = os.getcwd()

vendor_path = os.path.join(root_dir, 'vendor')
nbt_path = os.path.join(vendor_path, 'notebook-utils');
    
if not os.path.exists(nbt_path):
    if not os.path.exists(vendor_path):
        os.makedirs(vendor_path)
    %cd {vendor_path}
    !git clone https://github.com/duskvirkus/notebook-utils
    %cd {root_dir}
else:
    %cd {nbt_path}
    !git pull origin main
    %cd {root_dir}

## Variables

In [ ]:
input_folder = '/home/duskvirkus/dataset-creation/painterly-faces/raw/bj00100'
output_folder = '/home/duskvirkus/dataset-creation/painterly-faces/faces/bj00100'

## Setup

In [ ]:
import os
import sys
sys.path.insert(1, os.path.join(os.getcwd(), 'vendor/notebook-utils'))
from user_input import yes_or_no
from progress import update_progress
import shutil

%cd data/

dir_name = 'align-faces'

if not os.path.exists(dir_name):
    os.makedirs(dir_name)
else:
    if yes_or_no('Delete existing working directory and start fresh?'):
        shutil.rmtree(dir_name)
        os.makedirs(dir_name)
        
%cd {dir_name}

In [ ]:
if not os.path.exists('stylegan2-ada'):
    !git clone https://github.com/duskvirkus/stylegan2-ada
%cd stylegan2-ada
working_dir = os.getcwd()

In [ ]:
import os
import sys
import bz2
from tensorflow.keras.utils import get_file
from ffhq_dataset.face_alignment import image_align
from ffhq_dataset.landmarks_detector import LandmarksDetector

LANDMARKS_MODEL_URL = 'http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2'
RAW_IMAGES_DIR = input_folder
ALIGNED_IMAGES_DIR = output_folder

if not os.path.exists(ALIGNED_IMAGES_DIR):
        os.makedirs(ALIGNED_IMAGES_DIR)


def unpack_bz2(src_path):
    data = bz2.BZ2File(src_path).read()
    dst_path = src_path[:-4]
    with open(dst_path, 'wb') as fp:
        fp.write(data)
    return dst_path

landmarks_model_path = unpack_bz2(get_file('shape_predictor_68_face_landmarks.dat.bz2', LANDMARKS_MODEL_URL, cache_subdir='temp'))


landmarks_detector = LandmarksDetector(landmarks_model_path)

full_count = len([x for x in os.listdir(RAW_IMAGES_DIR) if x[0] not in '._'])
count = 0

for img_name in [x for x in os.listdir(RAW_IMAGES_DIR) if x[0] not in '._']:
    raw_img_path = os.path.join(RAW_IMAGES_DIR, img_name)
    for i, face_landmarks in enumerate(landmarks_detector.get_landmarks(raw_img_path), start=1):
        face_img_name = '%s_%02d.png' % (os.path.splitext(img_name)[0], i)
        aligned_face_path = os.path.join(ALIGNED_IMAGES_DIR, face_img_name)
        os.makedirs(ALIGNED_IMAGES_DIR, exist_ok=True)
        image_align(raw_img_path, aligned_face_path, face_landmarks, x_scale=1.2, em_scale=0.5)
    count += 1
    update_progress('align faces images progress', count / full_count)

update_progress('align faces images progress', 1)